In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import sys
sys.path.append('../')

import cv2
import pandas
import numpy as np
from tqdm import tqdm
from typing import Tuple

from src.utils.eye_tracking_data import get_eye_tracking_data
from src.config import SETS_PATH

OUTPUT_FILE_PATH = "../data/result.mp4"
FRAME_WIDTH = 600
FRAME_HEIGHT = 300
FPS = 144
CIRCLE_RADIUS = 5
CIRCLE_COLOR = (0, 0, 255, 100)

In [23]:
def get_background(
        experiment_id: int, 
        session_id: int, 
        sequence_id: int, 
        frame_width: int, 
        frame_height: int
    ) -> Tuple[np.ndarray, float | None]:
    background_file_path = f"{SETS_PATH}/experiment{experiment_id}"
    if experiment_id == 1 and session_id == 1:
        background_file_path += f"/images/scene{sequence_id}.png"
        background = cv2.imread(background_file_path)
        background = cv2.resize(background, (frame_width, frame_height))
        background_fps = None
    else:
        if experiment_id == 1 and session_id == 2:
            background_file_path += f"/videos/scene{sequence_id}.mp4"
        elif experiment_id == 2 and session_id == 1:
            background_file_path += f"/clear/scene{sequence_id}.mp4"
        elif experiment_id == 2 and session_id == 2:
            background_file_path += f"/overcast/scene{sequence_id}.mp4"
        video_capture = cv2.VideoCapture(background_file_path)
        background_fps = video_capture.get(cv2.CAP_PROP_FPS)
        background = []
        while True:
            ret, frame = video_capture.read()

            if not ret:
                break

            frame = cv2.resize(frame, (frame_width, frame_height))
            background.append(frame)
        video_capture.release()

    return background, background_fps

def visualize_gaze_sequence(
    experiment_id: int,
    session_id: int,
    participant_ids: int | None,
    sequence_id: int,
    output_file_path: str,
    frame_width: int,
    frame_height: int,
    fps: int,
    circle_radius: int,
    circle_color: Tuple[int, int, int],
):


    # Get data and group by single sequence experiment
    data = get_eye_tracking_data(
        experiment_id=experiment_id,
        session_id=session_id,
        participant_ids=participant_ids,
        sequence_ids=[sequence_id],
    )
    data = data.groupby(["ExperimentId", "SessionId", "ParticipantId", "SequenceId"])
    groups = [data.get_group(group) for group in data.groups]

    # Sort each sequence by timestamp and compute time difference between consecutive frames
    for i, group in enumerate(groups):
        group = group.copy()
        group = group.sort_values("Timestamp")
        group['TimeDiff'] = group['Timestamp'].diff().fillna(0)
        group['FrameDiff'] = group['TimeDiff'] / 1e7 * fps
        group['FrameNumber'] = group['FrameDiff'].cumsum().astype(int)
        group.drop(columns=["TimeDiff", "FrameDiff"], inplace=True)
        groups[i] = group


    # Get background image or video
    background, background_fps = get_background(experiment_id, session_id, sequence_id, frame_width, frame_height)

    # Initialize video writer
    fourcc = cv2.VideoWriter_fourcc('H','2','6','4')
    out = cv2.VideoWriter(output_file_path, fourcc, fps, (frame_width, frame_height))

    curr_frame = 0
    curr_group_coordinates_list = [None] * len(groups)
    next_frames = [group['FrameNumber'].iloc[0] for group in groups]
    max_frame = max([group['FrameNumber'].max() for group in groups])
    bar = tqdm(total=max_frame, desc="⌛ Generating gaze video...", unit="frames")
    while curr_frame < max_frame:

        # Get current background frame
        if isinstance(background, np.ndarray):
            frame = background.copy()
        else:
            curr_background_frame = int(curr_frame * background_fps / fps)
            curr_background_frame = min(curr_background_frame, len(background) - 1)
            frame = background[curr_background_frame].copy()

        # Update current gaze coordinates
        for i, group in enumerate(groups):
            next_frame = next_frames[i]
            if curr_frame == next_frame:
                # Update current gaze coordinates
                curr_group_coordinates = group[group['FrameNumber'] == curr_frame][['GazeX', 'GazeY']].values
                curr_group_coordinates_list[i] = curr_group_coordinates[0]

                # Update next frame
                next_frame = group[group['FrameNumber'] > curr_frame]['FrameNumber']
                if len(next_frame) > 0:
                    next_frame = next_frame.iloc[0]
                else:
                    next_frame = max_frame + 1 # Set to max frame to avoid updating gaze coordinates
                next_frames[i] = next_frame

        # Draw gaze coordinates
        for curr_group_coordinates in curr_group_coordinates_list:
            if curr_group_coordinates is not None:
                x = int(curr_group_coordinates[0] * frame_width)
                y = int(curr_group_coordinates[1] * frame_height)
                cv2.circle(frame, (x, y), radius=circle_radius, color=circle_color, thickness=-1)

        out.write(frame)
        curr_frame += 1
        bar.update(1)

    out.release()
    cv2.destroyAllWindows()

In [16]:
visualize_gaze_sequence(
    experiment_id=1,
    session_id=1,
    participant_ids=[1],
    sequence_id=2,
    output_file_path=OUTPUT_FILE_PATH, 
    frame_width=FRAME_WIDTH,
    frame_height=FRAME_HEIGHT,
    fps=FPS, 
    circle_radius=CIRCLE_RADIUS, 
    circle_color=CIRCLE_COLOR
)

⌛ Generating gaze video...: 100%|██████████| 12803/12803 [00:05<00:00, 2213.46frames/s]


In [24]:
visualize_gaze_sequence(
    experiment_id=1,
    session_id=2,
    participant_ids=None,
    sequence_id=1,
    output_file_path=OUTPUT_FILE_PATH, 
    frame_width=FRAME_WIDTH,
    frame_height=FRAME_HEIGHT,
    fps=FPS, 
    circle_radius=CIRCLE_RADIUS, 
    circle_color=CIRCLE_COLOR
)